This is a notebook that calculates neutron capture cross sections at a given T9 in units of millibarns $(mb)$

Here we import the relavent python packages

In [1]:
import sys
!{sys.executable} -m pip install --quiet wnutils

import sys
!{sys.executable} -m pip install --quiet astropy

import os, io, requests
from lxml import etree

import wnutils.xml as wx
import numpy as np
from astropy import constants as pc
from astropy import units as u

Now we download an xml that contains the reaction data

In [2]:
my_xml = wx.Xml(io.BytesIO(requests.get('https://osf.io/vqhxr/download').content))

In [3]:
t9 = 0.8
T = t9 * 1.e9 * u.K
Z = str(43)

In [4]:
v_t = np.sqrt((2 * pc.k_B.decompose(u.cgs.bases) * T) / pc.m_n.decompose(u.cgs.bases))

In [5]:
sig_dict = {}

filter = {}

In [6]:
nuclides = {}
nuc = my_xml.get_nuclide_data("[z = " + Z +"]")
nuclides.update(nuc)

We now specify that we only want $(n,\gamma)$ reactions

In [7]:
reac = my_xml.get_reaction_data("[reactant = 'n' and count(reactant) = 2  and product = 'gamma' and count(product) = 2]")

In [8]:
for s in reac:
    reaction = reac[s]
    rate = reaction.compute_rate(t9) * u.cm**3 /u.second /u.mol 
    sigma = (rate / (pc.N_A * v_t)).to(u.barn)
        
    usigma = 1000 * sigma / u.barn
    sig_dict[s] = usigma 

In [9]:
for reaction in reac:
    for reactant in reac[reaction].reactants:
        if reactant in nuclides:
            filter[reaction] = reaction

In [10]:
for key in list(sig_dict.keys()):
    if key not in filter:
        del sig_dict[key]

In [11]:
print('Cross Sections at T9 of ' + str(t9))

for k,v in sorted(sig_dict.items()):
    print(str(k)+': ' + str(v) + ' mb')

Cross Sections at T9 of 0.8
n + tc100 -> tc101 + gamma: 682.9014105179624 mb
n + tc101 -> tc102 + gamma: 171.89596373861357 mb
n + tc102 -> tc103 + gamma: 475.5629597706425 mb
n + tc103 -> tc104 + gamma: 140.10032671640542 mb
n + tc104 -> tc105 + gamma: 553.9022965260385 mb
n + tc105 -> tc106 + gamma: 71.99278573279153 mb
n + tc106 -> tc107 + gamma: 345.7871529023834 mb
n + tc107 -> tc108 + gamma: 78.9492252244812 mb
n + tc108 -> tc109 + gamma: 415.79436172877763 mb
n + tc109 -> tc110 + gamma: 88.38880231661076 mb
n + tc110 -> tc111 + gamma: 204.12059193577988 mb
n + tc111 -> tc112 + gamma: 44.428546227221574 mb
n + tc112 -> tc113 + gamma: 86.27031833144166 mb
n + tc113 -> tc114 + gamma: 12.364733418799595 mb
n + tc114 -> tc115 + gamma: 58.222037821514775 mb
n + tc115 -> tc116 + gamma: 8.335463053088931 mb
n + tc116 -> tc117 + gamma: 27.256580083741504 mb
n + tc117 -> tc118 + gamma: 5.409387545122122 mb
n + tc118 -> tc119 + gamma: 26.356062128409278 mb
n + tc119 -> tc120 + gamma: 2.945